<a href="http://cocl.us/pytorch_link_top">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/Pytochtop.png" width="750" alt="IBM Product " />
</a> 

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/cc-logo-square.png" width="200" alt="cognitiveclass.ai logo" />

<h1>Convolutional Neural Network with Data Augmentation </h1> 

<h2>Table of Contents</h2>
<p>In this lab, we train a Convolutional Neural Network with Regular data and Augmented data. We will show that the Augmented data imporoves improves generalization performance. </p>

<ul>
<li><a href="#Makeup_Data">Get Some Data</a></li>
<li><a href="#CNN">Convolutional Neural Network</a></li>
<li><a href="#R_training_data">Rotated Training Data</a></li>

</ul>
<p>Estimated Time Needed: <strong>25 min</strong> 14 min to train model </p>

<hr>

<h2>Preparation</h2>

In [ ]:
! wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/meet_up/12.02.2020/normal.pt

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/meet_up/12.02.2020/rotated_data.pt

In [1]:
pip install Pillow==6.2.2

    100% |████████████████████████████████| 3.9MB 8.5MB/s 
  Found existing installation: Pillow 5.4.1
    Uninstalling Pillow-5.4.1:
      Successfully uninstalled Pillow-5.4.1
Note: you may need to restart the kernel to use updated packages.


***Restart the kernel***

In [ ]:
# Import the libraries we need to use in this lab
# Using the following line code to install the torchvision library
# !conda install -y torchvision

import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pylab as plt
import numpy as np
import torchvision.datasets as dsets
import os  

Some useful functions for plotting 

plot cost and accuracy 

In [ ]:
def plot_cost_accuracy(checkpoint):

# Plot the cost and accuracy

    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(checkpoint['cost'], color=color)
    ax1.set_xlabel('epoch', color=color)
    ax1.set_ylabel('Cost', color=color)
    ax1.tick_params(axis='y', color=color)
    
    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color=color) 
    ax2.set_xlabel('epoch', color=color)
    ax2.plot( checkpoint['accuracy'], color=color)
    ax2.tick_params(axis='y', color=color)
    fig.tight_layout()

Define the function <code>show_data</code> to plot out data samples as images.

In [ ]:
def show_data(data_sample):
    plt.imshow(data_sample[0].numpy().reshape(IMAGE_SIZE, IMAGE_SIZE), cmap='gray')
    plt.title('y = '+ str(data_sample[1].item()))

Plot misclassified samples 

In [ ]:
def plot_mis_classified(model,dataset):
    count=0
    for x, y in torch.utils.data.DataLoader(dataset=dataset, batch_size=1):
        z = model(x)
        _, yhat = torch.max(z, 1)
        if yhat != y:
            show_data((x, y))
            plt.show()
            count += 1
        if count >= 5:
            break 

<!--Empty Space for separating topics-->

<h2 id="Makeup_Data">Load Data</h2> 

we create a transform object  <code> compose </code> one will resize the image and convert it to a tensor, the second will also rotate the image Randomly rotate the image.

In [ ]:
IMAGE_SIZE = 16

compose_rotate = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),transforms.RandomAffine(45), transforms.ToTensor()])
compose = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.ToTensor()])

Load the training dataset by setting the parameters <code>train </code> to <code>True</code>. We use the transform defined above, one with rotated data one without.

In [ ]:

train_dataset_rotate = dsets.MNIST(root='./data', train=True, download=True, transform=compose_rotate)
train_dataset = dsets.MNIST(root='./data', train=True, download=True, transform=compose)

Load the testing dataset by setting the parameters train  <code>False</code>, where the data is <b>ALL</b> rotated.

In [ ]:
# Make the validating 

validation_dataset = dsets.MNIST(root='./data', train=False, download=True, transform=compose_rotate)

We can see the data type is long.

In [ ]:
# Show the data type for each element in dataset

train_dataset[0][1].type()

Each element in the rectangular tensor corresponds to a number representing a pixel intensity as demonstrated by the following image.

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter%206/6.2.1imagenet.png" width="550" alt="MNIST data image">

Print out the fourth label 

In [ ]:
# The label for the fourth data element

train_dataset[3][1]

Plot the fourth sample 

In [ ]:
# The image for the fourth data element
show_data(train_dataset[3])

In [ ]:
show_data(train_dataset_rotate[3])

The fourth sample is a "1".

<!--Empty Space for separating topics-->

<h2 id="CNN">Build a Convolutional Neural Network Class</h2>

Build a Convolutional Network class with two Convolutional layers and one fully connected layer. Pre-determine the size of the final output matrix. The parameters in the constructor are the number of output channels for the first and second layer.

In [ ]:
class CNN(nn.Module):
    
    # Contructor
    def __init__(self, out_1=16, out_2=32):
        super(CNN, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=out_1, kernel_size=5, padding=2)
        self.maxpool1=nn.MaxPool2d(kernel_size=2)

        self.cnn2 = nn.Conv2d(in_channels=out_1, out_channels=out_2, kernel_size=5, stride=1, padding=2)
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(out_2 * 4 * 4, 10)
    
    # Prediction
    def forward(self, x):
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.maxpool1(x)
        x = self.cnn2(x)
        x = torch.relu(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x
    
    # Outputs in each steps
    def activations(self, x):
        #outputs activation this is not necessary
        z1 = self.cnn1(x)
        a1 = torch.relu(z1)
        out = self.maxpool1(a1)
        
        z2 = self.cnn2(out)
        a2 = torch.relu(z2)
        out1 = self.maxpool2(a2)
        out = out.view(out.size(0),-1)
        return z1, a1, z2, a2, out1,out

<h2 id="regular_data">Regular Data</h2> 

Define the Convolutional Neural Network Classifier, Criterion function, Optimizer and Train the Model

In [ ]:
# Create the model object using CNN class

model = CNN(out_1=16, out_2=32)

Plot the model parameters for the kernels before training the kernels. The kernels are initialized randomly.

Define the loss function, the optimizer and the dataset loader 

In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=5000)

This cell will train the model, we will comment it out as it takes a long time to run. You can load the trained model in the next cell.

<h3 id="Result">Analyze Results</h3> 

In [ ]:
checkpoint_normal = torch.load(os.path.join(os.getcwd(),'normal.pt'))
print("done")

plot accuracy and cost 

In [ ]:
plot_cost_accuracy(checkpoint_normal)

five misclassified samples

In [ ]:
model.load_state_dict(checkpoint_normal['model_state_dict'])
model.eval()

plot_mis_classified(model,validation_dataset)

<!--Empty Space for separating topics-->

<h2 id="R_training_data">Rotated Training Data</h2> 

Plot the loss and accuracy on the validation data:

In [ ]:
model_r = CNN(out_1=16, out_2=32)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model_r.parameters(), lr = learning_rate)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset_rotate, batch_size=100)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=5000)

This cell will train the model, we will comment it out as it takes a long time to run. You can load the trained model in the next cell.

<h3 id="Result">Analyze Results</h3> 

we load the checkpoint:

In [ ]:
checkpoint_rotated= torch.load(os.path.join(os.getcwd(),'rotated_data.pt'))
print("done")

plot accuracy and cost

In [ ]:
plot_cost_accuracy(checkpoint_rotated)

five misclassified samples

In [ ]:
model_r.load_state_dict(checkpoint_rotated['model_state_dict'])
model.eval()
plot_mis_classified(model_r,validation_dataset)

<!--Empty Space for separating topics-->

<a href="http://cocl.us/pytorch_link_bottom">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/notebook_bottom%20.png" width="750" alt="PyTorch Bottom" />
</a>

<h2>About the Authors:</h2> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

Other contributors: <a href="https://www.linkedin.com/in/michelleccarey/">Michelle Carey</a>, <a href="www.linkedin.com/in/jiahui-mavis-zhou-a4537814a">Mavis Zhou</a>

Thanks to Magnus <a href="http://www.hvass-labs.org/">Erik Hvass Pedersen</a> whose tutorials helped me understand convolutional Neural Network

<hr>

Copyright &copy; 2018 <a href="cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu">cognitiveclass.ai</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.